In [1]:
import os
import torch
import pandas as pd
import scanpy as sc
from sklearn import metrics
import multiprocessing as mp
from GraphST import GraphST

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_clusters = 10
random_seed=42
tool = 'mclust'

In [2]:
import numpy as np
import time
import tracemalloc
import anndata as ad
import gc

data_dir = '../../dataset/merfish_human_neocortex_SSL.h5ad'

adata = sc.read_h5ad(data_dir)
adata.X = adata.raw.X.copy()

del adata.raw
gc.collect()

32

In [3]:
import scipy.sparse as sp

saved_dir = './saved_matrices'
graph_neigh = sp.load_npz(f'{saved_dir}/graph_neigh.npz')
adj = sp.load_npz(f'{saved_dir}/adj.npz')

# Store only the required matrices
adata.obsm = {
    'graph_neigh': graph_neigh,
    'adj': adj,
    f'SSL_BT_ZS_{random_seed}': adata.obsm[f'SSL_BT_ZS_{random_seed}'],
    'spatial': adata.obsm['spatial']
}

ssl_adata = ad.AnnData(
    X = adata.obsm[f'SSL_BT_ZS_{random_seed}'],
    obs = adata.obs,
    obsm = adata.obsm,
)

del adata
gc.collect()


ssl_adata.var['highly_variable'] = [True] * ssl_adata.n_vars
print(f"Final matrix shapes: {ssl_adata.obsm['graph_neigh'].shape}")

Final matrix shapes: (234125, 234125)


In [4]:
GraphST.fix_seed(random_seed)

for slice_id in ssl_adata.obs['slice_id'].cat.categories:
    slice_mask = ssl_adata.obs['slice_id'] == slice_id 
    slice_indices = np.where(slice_mask)[0]
    current_adata = ssl_adata[slice_mask].copy()
    current_adata.obsm['graph_neigh'] = ssl_adata.obsm['graph_neigh'][slice_indices][:, slice_indices]
    current_adata.obsm['adj'] = ssl_adata.obsm['adj'][slice_indices][:, slice_indices]
    model = GraphST.GraphST(current_adata, device=device, epochs=100)
    current_adata = model.train()
    from GraphST.utils import clustering
    clustering(current_adata, n_clusters=n_clusters, method=tool)
    ssl_adata.obs.loc[slice_mask, 'mclust'] = current_adata.obs['mclust']
    del current_adata
    gc.collect()

Begin to train ST data...


  0%|                                                                                                        | 0/100 [00:00<?, ?it/s]

  1%|▉                                                                                               | 1/100 [00:00<00:38,  2.57it/s]

  3%|██▉                                                                                             | 3/100 [00:00<00:17,  5.46it/s]

  4%|███▊                                                                                            | 4/100 [00:00<00:18,  5.09it/s]

...

  |                                                                      |   0%

  |                                                                            

  |===================================                                   |  50%

  |                                                                            

  |======================================================================| 100%

In [5]:
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
from matplotlib import pyplot as plt
import os
# 结果保存目录
output_dir = '../human_neocortex_RM_ZS_GraphST_image'
os.makedirs(output_dir, exist_ok=True)
nmi_values = []
ari_values = []

def compute_nmi(adata, predict_key, ground_truth_key):
    return normalized_mutual_info_score(adata.obs[ground_truth_key], adata.obs[predict_key])

def compute_ari(adata, predict_key, ground_truth_key):
    return adjusted_rand_score(adata.obs[ground_truth_key], adata.obs[predict_key])

for batch in ssl_adata.obs['slice_id'].cat.categories:
    current_adata = ssl_adata[ssl_adata.obs['slice_id'] == batch]
    
    if 'tissue' in current_adata.obs:
        nmi = compute_nmi(adata=current_adata, predict_key='mclust', ground_truth_key='tissue')
        ari = compute_ari(adata=current_adata, predict_key='mclust', ground_truth_key='tissue')
        nmi = np.round(nmi, 3)
        ari = np.round(ari, 3)

        nmi_values.append(nmi)
        ari_values.append(ari)

        title = f'{batch}\n NMI: {nmi}  ARI: {ari}'
        print(title)
    else:
        title = f'{batch}'

    axes = sc.pl.embedding(adata=current_adata, basis='spatial', color='mclust', show=False)
    axes.axis('equal')
    axes.set_title(f'{title} (GraphST)')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'GraphST_BT_ZS_{random_seed}_embedding_{batch}.png'), dpi=400)
    plt.savefig(os.path.join(output_dir, f'GraphST_BT_ZS_{random_seed}_embedding_{batch}.eps'), format='eps', dpi=400)
    plt.close()
    # 保存到当前目录下名为 results 的文件夹中
    # 当前 Notebook 文件名和实验名称
    notebook_name = "barlow_twins_zero_shot_GraphST_MERSCOPE_human_neocortex_42.ipynb"
    experiment_name = "barlow_twins_zero_shot_GraphST_MERSCOPE_human_neocortex_42"

    if not os.path.exists('results'):
        os.makedirs('results')

    # 定义 CSV 文件的路径
    csv_filename = f"results/{experiment_name}_results.csv"

    # 保存 Batch, NMI, ARI, 和 experiment_name 到 CSV 文件中
    import csv
    with open(csv_filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        # 如果文件是新建的，写入表头
        if os.stat(csv_filename).st_size == 0:
            writer.writerow(['Experiment Name', 'Batch', 'NMI', 'ARI'])
        writer.writerow([experiment_name, batch, nmi, ari])


ARKFrozen-62-PFC
 NMI: 0.583  ARI: 0.51


/home/hanchuangyi/miniconda3/envs/stagate_pyG/lib/python3.10/site-packages/scanpy/plotting/_utils.py:487: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[value_to_plot + "_colors"] = colors_list


NIH-4365-BA10
 NMI: 0.516  ARI: 0.477


/home/hanchuangyi/miniconda3/envs/stagate_pyG/lib/python3.10/site-packages/scanpy/plotting/_utils.py:487: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[value_to_plot + "_colors"] = colors_list


NIH-4392-BA17
 NMI: 0.284  ARI: 0.212


/home/hanchuangyi/miniconda3/envs/stagate_pyG/lib/python3.10/site-packages/scanpy/plotting/_utils.py:487: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[value_to_plot + "_colors"] = colors_list


NIH-5900-BA17
 NMI: 0.401  ARI: 0.327


/home/hanchuangyi/miniconda3/envs/stagate_pyG/lib/python3.10/site-packages/scanpy/plotting/_utils.py:487: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[value_to_plot + "_colors"] = colors_list


UCSF2018-003-MFG
 NMI: 0.433  ARI: 0.286


/home/hanchuangyi/miniconda3/envs/stagate_pyG/lib/python3.10/site-packages/scanpy/plotting/_utils.py:487: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[value_to_plot + "_colors"] = colors_list


In [6]:
new_adata = sc.read_h5ad(data_dir)
new_adata.obs[f'GraphST_BT_ZS_{random_seed}'] = ssl_adata.obs['mclust']
new_adata

AnnData object with n_obs × n_vars = 234125 × 300
    obs: 'Cell_ID', 'orig.ident', 'nCount_Vizgen', 'nFeature_Vizgen', 'z', 'volume', 'fov', 'nCount_SCT', 'nFeature_SCT', 'seurat_clusters', 'Sample_ID', 'Estimated_postconceptional_age_in_days', 'Group', 'Region', 'Region_details', 'integrated_snn_res.2.8', 'class', 'subclass', 'type', 'nCount_niche', 'nFeature_niche', 'niches', 'niche_name', 'type_updated', 'x', 'y', 'cell_type', 'tissue', 'slice_id', 'cellcharter_scVI', 'cellcharter_SSL_BT_ZS', 'cellcharter_SSL_RM_ZS', 'STAGATE_SSL_BT_ZS', 'STAGATE_SSL_RM_ZS', 'STAGATE_original', 'cellcharter_SSL_BT_FT', 'cellcharter_SSL_RM_FT', 'STAGATE_SSL_BT_FT', 'STAGATE_SSL_RM_FT', 'cellcharter_BT_FT_42', 'STAGATE_RM_FT_42', 'STAGATE_BT_FT_42', 'STAGATE_BT_FT_761', 'STAGATE_BT_FT_9824', 'STAGATE_BT_ZS_476', 'STAGATE_BT_ZS_3407', 'STAGATE_RM_FT_761', 'STAGATE_RM_ZS_42', 'STAGATE_RM_ZS_476', 'STAGATE_RM_ZS_761', 'STAGATE_RM_ZS_3407', 'STAGATE_RM_ZS_9824', 'STAGATE_BT_FT_476', 'STAGATE_BT_FT_3407',

In [7]:
new_adata.write_h5ad(data_dir)